In [19]:
import pandas as pd
import numpy as np

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
df = pd.read_excel('../dataset/main.xlsx')
df.shape

(36791, 18)

In [22]:
#setup y(target) dan x(fitur)
X = df[['likes','dislikes','comment_count','No_tags']] #fitur dapat disesuaikan
y = df[['views']] #target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [23]:
#standarisai scale
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

numerical_features = X.columns.to_list()
for n in numerical_features:
  scaler = ss.fit(X_train[[n]])
  X_train[n] = scaler.transform(X_train[[n]])
  X_test[n] = scaler.transform(X_test[[n]])
  # scaler.to_pickle(filename)

In [24]:
 #Train linear regression
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

In [25]:
X_test.head()

,likes,dislikes,comment_count,No_tags
3792,-0.255827,-0.084666,-0.157930,0.305587
13480,-0.245546,-0.030243,-0.103442,-1.123779
21110,-0.260685,-0.080508,-0.168963,-1.225877
35549,-0.246378,-0.096208,-0.141243,-0.000706
2952,-0.275361,-0.091802,-0.178567,0.101392


In [26]:
print(regressor.intercept_)

[1071162.8837417]


In [27]:
y_pred

array([[419019.01007475],
       [296382.45266886],
       [241060.84068646],
       ...,
       [527398.9309909 ],
       [349493.90853867],
       [329109.05832755]])

In [28]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25753 entries, 23009 to 15795
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   likes          25753 non-null  float64
 1   dislikes       25753 non-null  float64
 2   comment_count  25753 non-null  float64
 3   No_tags        25753 non-null  float64
dtypes: float64(4)
memory usage: 1006.0 KB


In [31]:
X.columns

Index(['likes', 'dislikes', 'comment_count', 'No_tags'], dtype='object')

In [29]:
regressor.coef_

array([[2765212.87623676,  696126.82089744, -504607.69717225,
         112955.50108942]])

In [30]:
#check overfit/underfit
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import cross_validate

def eval_regression(model):
  y_pred = model.predict(X_test)
  y_pred_train = model.predict(X_train)

  print('RMSE (test): ' + str(mean_squared_error(y_test, y_pred, squared=False)))
  print('RMSE (train): ' + str(mean_squared_error(y_train, y_pred_train, squared=False)))

  print('MAPE (test): ' + str(mean_absolute_percentage_error(y_test, y_pred)))
  print('MAPE (train): ' + str(mean_absolute_percentage_error(y_train, y_pred_train)))

  print('r2 (test): ' + str(r2_score(y_test, y_pred)))
  print('r2 (train): ' + str(r2_score(y_train, y_pred_train)))

  # k-fold cross validation
  score = cross_validate(model, X, y, cv=5, scoring='r2', return_train_score=True)
  print('r2 (cross-val test): ' + str(score['test_score'].mean()))
  print('r2 (cross-val train): ' + str(score['train_score'].mean()))

eval_regression(regressor)

RMSE (test): 1634492.3467801376
RMSE (train): 1587665.0654531745
MAPE (test): 1.8277406284538156
MAPE (train): 1.9465243969587949
r2 (test): 0.7476042361864546
r2 (train): 0.7518341686010438
r2 (cross-val test): 0.5250221741715304
r2 (cross-val train): 0.7539354988423217
